In [15]:
import os
import itertools

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

import random

from os import listdir
from os.path import isfile, join
import pickle

plt.style.use('seaborn-white')

%matplotlib inline

from scipy.stats import gamma, poisson

import epyestim
import epyestim.covid19 as covid19
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE, r2_score
from xgboost import XGBRegressor, DMatrix, train
from sklearn.multioutput import MultiOutputRegressor

from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from pykalman import KalmanFilter

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, TimeDistributed, RepeatVector
from keras.callbacks import EarlyStopping
import keras

import plotly.express as px
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

from functools import partial
import copy


to_sum_KPIs = ['totale_casi_giornalieri', 'terapia_intensiva_giornalieri', 'terapia_intensiva', 'nuovi_positivi', 'tamponi_giornalieri']
covidKPIsPrecompute = ['%pos']+to_sum_KPIs
trafficKPIsPrecompute = ['Handover', 'Download vol.', 'Upload vol.', '#Users']

data_path = "/Users/filipkrasniqi/Documents/Datasets.tmp/traffic-covid/"
saved = "{}saved/".format(data_path)
traffic_daily = "{}TS_1800_daily.pkl".format(saved)
region_traffic_daily = "{}all.pkl".format(saved)
covid = "{}covid/".format(data_path)
covid_daily = "{}covid_2303.csv".format(covid)

capped_last_date = pd.to_datetime('2021-01-15')

In [16]:
dfs = []
path_covid_predictions="{}predictions/covid_2303.pkl".format(saved)
recompute_rt = False
if recompute_rt:
    for r in regions:
        print("REGIONE: {}".format(r))
        current_df = df_covid.loc[df_covid['Regione'] == r]
        current_df['Date'] = pd.to_datetime(current_df['Date']).dt.date
        current_df['DateIndex'] = current_df.loc[:, 'Date']
        current_df.set_index('DateIndex', inplace=True)
        #current_df = current_df.loc[current_df['nuovi_positivi'] > 0]
        current_df = current_df.loc[pd.to_datetime('2020/03/01'):]
        idxs = (current_df['nuovi_positivi'] < 0)# | (current_df.isna()) | (current_df['nuovi_positivi'] == np.inf) | (current_df['nuovi_positivi'] == -np.inf)
        if idxs.sum() > 0:
            current_df.loc[idxs, 'nuovi_positivi'] = np.nan
        current_df.fillna(method='ffill', inplace=True)
        current_df.dropna(subset=['nuovi_positivi'], inplace=True)
        #current_df[current_df.loc[:, 'nuovi_positivi']]
        #current_df.dropna(subset=['nuovi_positivi'], inplace=True)
        #
        current_df = current_df.drop_duplicates(keep='first')
        #print(current_df['nuovi_positivi'].shape, current_df['nuovi_positivi'].apply(lambda x: x < 0).sum())
        #current_df.dropna(subset=['totale_casi_giornalieri'], inplace=True)
        #print(current_df['totale_casi_giornalieri'].isna().sum())
        #print(current_df['totale_casi_giornalieri'].sum())
        r_t_series = covid19.r_covid(current_df['nuovi_positivi'])
        current_df = pd.merge(current_df, r_t_series, left_index=True, right_index=True)
        dfs.append(current_df)
    df_covid_predictions = pd.concat(dfs)
    del dfs
    df_covid_predictions.set_index(['Date', 'Regione'], inplace=True)
    df_covid_predictions['%pos'] = (df_covid_predictions['nuovi_positivi']/df_covid_predictions['tamponi_giornalieri'])
    df_covid_predictions.to_pickle(path_covid_predictions)
else:
    df_covid_predictions = pd.read_pickle(path_covid_predictions)

df_unseen_covid = df_covid_predictions.loc[df_covid_predictions.index.get_level_values('Date')>=capped_last_date]
#df_covid_predictions = df_covid_predictions.loc[df_covid_predictions.index.get_level_values('Date')<capped_last_date]

In [17]:
# TODO importare modelli
def init_models_dict(model_dir):
    path_results_dir = join(saved, 'models', model_dir)
    models = os.listdir(path_results_dir)
    models_dict = {}
    #model_path = join(path_results_dir, model)
    regions = os.listdir(path_results_dir)
    for region in regions:
        region_path = join(path_results_dir, region)
        model = keras.models.load_model(region_path)
        models_dict[region]=model
        #model_names = sorted([f for f in os.listdir(region_path) if isfile(join(region_path, f))], key=lambda f: int(f.split("_")[1].split(".")[0]))
    return models_dict

In [56]:
def init_datasets(to_check):
    model_dir = to_check+"_models"#'results_v16_ma_poly_after_update_only_covid_models'

    path_datasets = "{}predictions/{}".format(saved, to_check)
    #path_train = "{}_df_train".format(path_datasets)
    path_test = "{}_df_test.csv".format(path_datasets)
    path_unseen = "{}_df_unseen.csv".format(path_datasets)

    df_test_selected = pd.read_csv(path_test)
    df_test_selected['Date'] = pd.to_datetime(df_test_selected['Date'])
    df_test_selected.set_index(['Date', 'Regione'], inplace=True)
    
    df_unseen_selected = pd.read_csv(path_unseen)
    df_unseen_selected['Date'] = pd.to_datetime(df_unseen_selected['Date'])
    df_unseen_selected.set_index(['Date', 'Regione'], inplace=True)
    
    current_models_regions = init_models_dict(model_dir)
    return df_test_selected, df_unseen_selected, current_models_regions

def figure_from_name_region_model(fig, to_check, region, farsightness, row):
    df_test_selected, df_unseen_selected, current_models_regions = init_datasets(to_check)

    current_model = current_models_regions[region]

    num_samples_retrain = 64

    last_date_seen = df_test_selected.index.get_level_values('Date').max()
    start_date_unseen = last_date_seen+datetime.timedelta(days=1)

    current_region_df = df_unseen_selected.xs(region, level='Regione')
    current_region_test_df = df_test_selected.xs(region, level='Regione')

    current_train, current_unseen = current_region_test_df, current_region_df

    # take only last num_samples_retrain
    current_train = current_train.iloc[-num_samples_retrain:]
    features_selected = [col for col in current_train.columns if "target" not in col]

    features_train = current_train[features_selected]
    features_unseen = current_unseen[features_selected]

    num_features_t = len([col for col in features_selected if not any(char.isdigit() for char in col)])
    n_timesteps = features_train.shape[1]//num_features_t

    dates = features_unseen.index
    num_dates= len(dates)
    predictions, index_predictions = [], []
    idx_dates = list(range(0, num_dates, farsightness))
    dates_to_return = dates[idx_dates]

    min_y, max_y = float("+inf"), float("-inf")

    for idx, idx_d in enumerate(idx_dates):
        d = dates[idx_d]
        print("TEST {}".format(d))
        feature_current = features_unseen.loc[d:d]#.to_numpy()
        #print(feature_current)
        feature_current = feature_current.to_numpy().reshape(-1, n_timesteps, num_features_t, order='C')
        #print(feature_current.shape)
        current_predictions = current_model.predict(feature_current).flatten()[:farsightness]
        print("DBG: {}".format(current_predictions.shape))
        for p in current_predictions:
            predictions.append(p)
        next_d = d + datetime.timedelta(days=farsightness)
        for d in [d for d in pd.date_range(d, next_d)]:
            index_predictions.append(d)
        
        if idx < len(idx_dates)-1:
            print("RETRAIN")
            # I can retrain
            current_train_df = current_region_df.loc[:next_d]

            current_train_df = current_train_df.iloc[-num_samples_retrain:]
            current_features_train = current_train_df[features_selected]

            #for key in model_per_target.keys():
            #model = model_per_target[key]
            features_at_day_d = current_region_df.loc[:d]
            target_series_train = current_train_df[[col for col in current_train_df.columns if "target" in col]]
            current_features_train, target_series_train = current_features_train.dropna(), target_series_train.dropna()
            common_idxs = current_features_train.index.intersection(target_series_train.index)
            current_features_train = current_features_train.loc[common_idxs]
            target_series_train = target_series_train.loc[common_idxs]

            size_int = int(current_features_train.shape[0]*0.7)
            current_features_train_1, current_features_train_2 = current_features_train.iloc[:size_int], current_features_train.iloc[size_int:]
            current_target_train, current_target_val = target_series_train.loc[current_features_train_1.index], target_series_train.loc[current_features_train_2.index]
            lstm_input_train = current_features_train_1.to_numpy().reshape(-1, n_timesteps, num_features_t, order='C')
            lstm_input_val = current_features_train_2.to_numpy().reshape(-1, n_timesteps, num_features_t, order='C')
            
            MAX_EPOCHS = 128
            early_stopping = EarlyStopping(monitor='val_loss', patience=3, mode='min', restore_best_weights=True)
            
            if lstm_input_train.shape[0] > 0:
                history = current_model.fit(x=lstm_input_train, y=current_target_train, epochs=MAX_EPOCHS, verbose=0,
                                  validation_data=(lstm_input_val, current_target_val),
                                  callbacks=[early_stopping])
    
    predictions = np.array(predictions).flatten()
    print("Predictions ({}) -> {}".format(len(predictions), predictions))
    print("Indices ({}) -> {}".format(len(index_predictions), index_predictions))
    
    index_predictions = index_predictions[:len(predictions)]
    series_predictions = pd.Series(data=predictions, index=index_predictions)
    series_target = df_unseen_covid.xs(region, level='Regione').loc[start_date_unseen:]['R_mean']


    min_y = min(series_predictions.min(), series_target.min())
    max_y = max(series_predictions.max(), series_target.max())

    for d in [dates[i] for i in idx_dates]:
        fig.add_trace(go.Scatter(
            line = dict(color="black", width=2, dash='dash'),
            marker_size=1,
            x=[d, d],
            y=[min_y, max_y],
            showlegend=False
        ), row=row, col=1)       

    #series_target.plot()
    #series_predictions.plot()


    fig.add_trace(go.Scatter(
            line = dict(color="orange", width=1),
            marker_size=1,
            x=series_predictions.index,
            y=series_predictions,
            showlegend=False
        ), row=row, col=1)

    fig.add_trace(go.Scatter(
            line = dict(color="red", width=1),
            marker_size=1,
            x=series_target.index,
            y=series_target,
            showlegend=False
        ), row=row, col=1)
    return fig

In [57]:
cache_models = False
if cache_models:
    to_check = 'results_v5_LSTM'
    to_check_both, to_check_traffic, to_check_rmean = "{}_both".format(to_check), "{}_only_traffic".format(to_check), "{}_only_rmean".format(to_check)
    to_check_traffic_no_mobility = "{}_only_traffic_no_mobility".format(to_check)

    df_test_selected, df_unseen_selected, _ = init_datasets(to_check_both)

In [58]:
name = "Compare input on unseen data"
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app_results = JupyterDash(name, external_stylesheets=external_stylesheets)

all_regions = df_test_selected.index.get_level_values('Regione').unique()

app_results.layout = html.Div([
html.Label(
    [
        "Regione",
        dcc.Dropdown(id="region",
                     options=[{"label": x, "value": x} for x in all_regions],
                    value="Lombardia",
                    clearable=False)
    ]),

html.Label(
    [
        "Farsightness",
        dcc.Dropdown(id="farsightness",
                     options=[{"label": x, "value": x} for x in [9, 17, 27, 34]],
                    value=9,
                    clearable=False)
    ]),

html.Label(
            ["Rolling average window ",
            html.Br(),
            dcc.Input(
                id='rolling_avg',
                type='number',
                value=1
            )]
),
html.Div(dcc.Graph(id=name))])

@app_results.callback(
Output(name, "figure"), 
[Input("region", "value"), Input("farsightness", "value"), Input("rolling_avg", "value")])
def display_rf_results(region, farsightness, roll_avg):
    
    fig = make_subplots(vertical_spacing = 0.01, horizontal_spacing = 0.01, rows=4, cols=1, \
                    shared_xaxes=True, shared_yaxes=True, row_titles=['All', 'T+M', 'Only T', 'Only C'], column_titles=['Unseen: target vs prediction'])#subplot_titles=subplot_titles)

    fig = figure_from_name_region_model(fig, to_check_both, region, farsightness, 1)
    #fig = figure_from_name_region_model(fig, to_check_traffic, region, model_name, farsightness, 2)
    #fig = figure_from_name_region_model(fig, to_check_traffic_no_mobility, region, model_name, farsightness, 3)
    #fig = figure_from_name_region_model(fig, to_check_rmean, region, model_name, farsightness, 4)
    
    
    return fig

#app_timeseries = build_app_timeseries(df_traffic_daily_SO, df_covid_SO)
app_results.run_server(mode='inline', port=49008) # debug=True, use_reloader=False

TEST 2021-01-16 00:00:00
DBG: (9,)
RETRAIN
TEST 2021-01-25 00:00:00
DBG: (9,)
RETRAIN
TEST 2021-02-03 00:00:00
DBG: (9,)
RETRAIN
TEST 2021-02-12 00:00:00
DBG: (9,)
Predictions (36) -> [0.85292673 0.8591075  0.86302966 0.8705704  0.8940509  0.926978
 0.95881134 0.9849428  1.0033805  1.0043985  1.0269228  0.99807334
 0.97245604 0.94707704 0.91690713 0.9025933  0.8930854  0.88669324
 1.0184944  1.0486392  1.0396134  1.0188608  1.0056682  0.9884053
 0.96393734 0.9458586  0.942184   1.0479251  1.051803   1.0391144
 1.0166308  1.005773   0.9957253  0.98209417 0.96584016 0.96366036]
Indices (40) -> [Timestamp('2021-01-16 00:00:00', freq='D'), Timestamp('2021-01-17 00:00:00', freq='D'), Timestamp('2021-01-18 00:00:00', freq='D'), Timestamp('2021-01-19 00:00:00', freq='D'), Timestamp('2021-01-20 00:00:00', freq='D'), Timestamp('2021-01-21 00:00:00', freq='D'), Timestamp('2021-01-22 00:00:00', freq='D'), Timestamp('2021-01-23 00:00:00', freq='D'), Timestamp('2021-01-24 00:00:00', freq='D'), Time